# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2.
3. 
4. 

## Import

In [1]:
import numpy as np
import pandas as pd

## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

## E. Tổng hợp lại quá trình thực hiện đồ án